In [127]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


In [128]:
# me traigo el ds con las columnas basicas
df_movies= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\A-ETLconsignas\MoviesSMALL.csv')
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44936 entries, 0 to 44935
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             44936 non-null  float64
 1   id                 44936 non-null  int64  
 2   original_language  44926 non-null  object 
 3   overview           44029 non-null  object 
 4   popularity         44936 non-null  float64
 5   release_date       44936 non-null  object 
 6   revenue            44936 non-null  float64
 7   runtime            44696 non-null  float64
 8   title              44936 non-null  object 
 9   release_year       44936 non-null  float64
 10  return             10879 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 3.8+ MB


In [129]:
# transformo la columna overview toda en tipo str
df_movies["overview"] = df_movies["overview"].astype(str)

In [130]:
df_movies['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [131]:
#elimino las columnas que no voy a usar, me quedo unicamente con popularity, title y id
columnas_elim=['budget','release_year', 'release_date', 'runtime', 'revenue', 'return','original_language']
df_movies2 = df_movies.drop(columns=columnas_elim)
df_movies_sorted= df_movies2.sort_values(ascending=False, by='popularity')

In [132]:
# me quedo con las 15 mil peliculas mas populares
df_top_5= df_movies_sorted.head(15000) 

In [133]:
df_top_5.head()

,id,overview,popularity,title
30443,211672,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,Minions
33060,297762,An Amazon princess comes to the world of Man t...,294.337037,Wonder Woman
41779,321612,A live-action adaptation of Disney's version o...,287.253654,Beauty and the Beast
43170,339403,After being coerced into working for a crime b...,228.032744,Baby Driver
24267,177572,The special bond that develops between plus-si...,213.849907,Big Hero 6


In [134]:
#hago un csv
df_top_5.to_csv('Top15kpelis.csv', index=False)

In [135]:
# me traido el data set de genero y id
df_genero=pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\DESANIDANDO\GenerosAgrupados.csv') 
df_genero.head()

,id,name_genres
0,2,"Drama,Crime"
1,3,"Drama,Comedy"
2,5,"Crime,Comedy"
3,6,"Action,Thriller,Crime"
4,11,"Adventure,Action,Science Fiction"


In [136]:
# me traigo el dataset de cast
df_cast= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\DESANIDANDO\ActoresPorPEliID.csv')
df_cast.rename(columns={"id_original": "id"}, inplace=True)
df_cast.drop(columns='name', inplace=True)
df_cast.head()

,id,character
0,2,"Turo Pajala, Susanna Haavisto, Matti Pellonpä..."
1,3,"Matti Pellonpää, Kati Outinen, Sakari Kuosman..."
2,5,"Tim Roth, Antonio Banderas, Jennifer Beals, M..."
3,6,"Emilio Estevez, Cuba Gooding Jr., Denis Leary..."
4,11,"Mark Hamill, Harrison Ford, Carrie Fisher, Pe..."


In [137]:
# me traigo el dataset de directores
df_directores= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\DESANIDANDO\directoresAgrupadosID.csv')
df_directores.rename(columns={"id_original": "id"}, inplace=True)
df_directores.head()

,id,name
0,100,Guy Ritchie
1,10000,Sergio Cabrera
2,10001,Yahoo Serious
3,100010,Frank Borzage
4,100017,Angelina Maccarone


In [138]:
# uno los dataset de los top 15mil con genero, cast y directores
df_concat= pd.merge(df_top_5,df_cast, on='id', how='left')
df_concat2= pd.merge(df_concat, df_directores, on='id', how ='left')
df_concat3= pd.merge(df_concat2, df_genero, on='id', how='left')


In [139]:
#este data set me va a servir para la funcion de ML
df_concat3.head()


,id,overview,popularity,title,character,name,name_genres
0,211672,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,Minions,"Sandra Bullock, Jon Hamm, Michael Keaton, All...","Kyle Balda,Pierre Coffin","Family,Animation,Adventure,Comedy"
1,297762,An Amazon princess comes to the world of Man t...,294.337037,Wonder Woman,"Gal Gadot, Chris Pine, Robin Wright, Danny Hu...",Patty Jenkins,"Action,Adventure,Fantasy"
2,321612,A live-action adaptation of Disney's version o...,287.253654,Beauty and the Beast,"Emma Watson, Dan Stevens, Luke Evans, Kevin K...",Bill Condon,"Family,Fantasy,Romance"
3,339403,After being coerced into working for a crime b...,228.032744,Baby Driver,"Ansel Elgort, Lily James, Kevin Spacey, Jamie...",Edgar Wright,"Action,Crime"
4,177572,The special bond that develops between plus-si...,213.849907,Big Hero 6,"Scott Adsit, Ryan Potter, Daniel Henney, T.J....","Chris Williams,Don Hall","Adventure,Family,Animation,Action,Comedy"


In [140]:
df_concat3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           15000 non-null  int64  
 1   overview     15000 non-null  object 
 2   popularity   15000 non-null  float64
 3   title        15000 non-null  object 
 4   character    15000 non-null  object 
 5   name         14978 non-null  object 
 6   name_genres  14982 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 820.4+ KB


In [141]:
# me aseguro de que todas las columnas sean str
for column in df_concat3.columns:
    df_concat3[column] = df_concat3[column].astype(str)

In [142]:
#la pongo en un DS
df_concat3.to_csv('FuncionML.csv', index=False)

In [143]:
###Ahora elimindo las columnas numericas
elim= ['id','popularity']
filters= df_concat3.drop(columns=elim)
filters.head()

,overview,title,character,name,name_genres
0,"Minions Stuart, Kevin and Bob are recruited by...",Minions,"Sandra Bullock, Jon Hamm, Michael Keaton, All...","Kyle Balda,Pierre Coffin","Family,Animation,Adventure,Comedy"
1,An Amazon princess comes to the world of Man t...,Wonder Woman,"Gal Gadot, Chris Pine, Robin Wright, Danny Hu...",Patty Jenkins,"Action,Adventure,Fantasy"
2,A live-action adaptation of Disney's version o...,Beauty and the Beast,"Emma Watson, Dan Stevens, Luke Evans, Kevin K...",Bill Condon,"Family,Fantasy,Romance"
3,After being coerced into working for a crime b...,Baby Driver,"Ansel Elgort, Lily James, Kevin Spacey, Jamie...",Edgar Wright,"Action,Crime"
4,The special bond that develops between plus-si...,Big Hero 6,"Scott Adsit, Ryan Potter, Daniel Henney, T.J....","Chris Williams,Don Hall","Adventure,Family,Animation,Action,Comedy"


In [144]:
filters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   overview     15000 non-null  object
 1   title        15000 non-null  object
 2   character    15000 non-null  object
 3   name         15000 non-null  object
 4   name_genres  15000 non-null  object
dtypes: object(5)
memory usage: 586.1+ KB


In [145]:
# Me aseguro que todas las columnas sean str para que no me de error la funcion
for column in filters.columns:
    filters[column] = filters[column].astype(str)

In [146]:
#elimino espacios entre texto y mayusculas
def clean_data(x):
    return str.lower(x.replace(" ",""))


In [147]:
#aplico la funcion
for column in filters.columns:
    filters[column] = filters[column].apply(clean_data)


In [148]:
filters.head()

,overview,title,character,name,name_genres
0,"minionsstuart,kevinandbobarerecruitedbyscarlet...",minions,"sandrabullock,jonhamm,michaelkeaton,allisonjan...","kylebalda,pierrecoffin","family,animation,adventure,comedy"
1,anamazonprincesscomestotheworldofmantobecometh...,wonderwoman,"galgadot,chrispine,robinwright,dannyhuston,dav...",pattyjenkins,"action,adventure,fantasy"
2,alive-actionadaptationofdisney'sversionofthecl...,beautyandthebeast,"emmawatson,danstevens,lukeevans,kevinkline,jos...",billcondon,"family,fantasy,romance"
3,"afterbeingcoercedintoworkingforacrimeboss,ayou...",babydriver,"anselelgort,lilyjames,kevinspacey,jamiefoxx,jo...",edgarwright,"action,crime"
4,thespecialbondthatdevelopsbetweenplus-sizedinf...,bighero6,"scottadsit,ryanpotter,danielhenney,t.j.miller,...","chriswilliams,donhall","adventure,family,animation,action,comedy"


Ahora creamos un conjunto de metadatos

In [149]:
def crear_datos(x):
    return x['name'] + ' ' + x['character']+ ' ' + x['name_genres']+ ' ' + x['overview']


In [150]:
filters['metadatos']= filters.apply(crear_datos, axis=1)

In [151]:
filters.isnull().mean()

overview       0.0
title          0.0
character      0.0
name           0.0
name_genres    0.0
metadatos      0.0
dtype: float64

In [152]:
filters.to_csv('filters.csv', index=False)

In [153]:
##usamos Count Vectorizer para convertir las palabras en vectores
from sklearn.feature_extraction.text import CountVectorizer
#elimino palabras que no me sirven
count= CountVectorizer(stop_words='english')
count_matrix= count.fit_transform(filters['metadatos'])

In [154]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2= cosine_similarity(count_matrix, count_matrix)

In [155]:
##Reestrablezco el indice de Df inicial y hago le mapeo inverso
filters= filters.reset_index()
indices=pd.Series(filters.index, index=filters['title'])

In [ ]:
## achico el dataset a las 5 mil mas populares, para que no demore tanto la funcion y no consuma tanta memoria
df_top_4= df_top_5.head(5000)
##creo el dataset, para luego ponerlo en la carpeta junto con el main
df_top_4.to_csv('Funcion4milpelis.csv')

In [156]:
filters2= pd.read_csv('filters2.csv')
df_top_4=pd.read_csv('Funcion4milpelis.csv')

In [157]:
def recomendacion(title):
    '''Ingresa un nombre de película y devuelve una lista de películas similares basadas en sus resúmenes'''
    
    # instancia de CountVectorizer para convertir los textos en vectores numéricos 
    count = CountVectorizer(stop_words='english') # palabras vacias o comunes en el idioma indicado como english .
    count_matrix = count.fit_transform(filters2['metadatos']) # utilizamos la columna metadatos de nuestro dataframe filters2, y matriziamos el conteo de cada palabra
    
    # Calcularemos la similitud del coseno entre los vectores que representan los metadatos de las películas en filters.
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    
    # Convertir el título a minúsculas y eliminar espacios en blanco
    title = title.replace(' ', '').lower() 
    
    # índice de la película en el df_top_4
    if title not in indices: # esto verifica si el nombre ingresado se encuentra en la columna title
        return {'Mensaje': 'Película no encontrada'} # si no la encuentra sale este mensaje
    
    idx = df_top_4[df_top_4['title'].str.replace(' ', '').str.lower() == title].index[0] #  busca el índice correspondiente de la película en el DataFrame df_top_5
    
    # nos indica los puntajes de similitud y los ordena.
    sim_scores = list(enumerate(cosine_sim[idx])) # aqui esta obteniendo los puntajes de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # posteriomente los estamos ordenando. El lambda usa como entrada (x) y devuelve el valor de similitud (x[1])
    # ponemo el reverse true porque ordena en orden descendente según el valor de similitud, valga la redundancia.



    #Para que devuelva solamente las pelis unicamente que indiquemos, si elimino esta linea devuelve una lista mucho mayor de peliculas similares.
    sim_scores = sim_scores[1:6] #[1:6] para que devuelva solamente 5
    
    # índices de las películas similares (excluyendo la película consultada)
    similar_movie_indices = [i[0] for i in sim_scores if i[0] != idx]
    # Tomaremos cada elemento 'i' en la lista 'sim_scores'.
    # extrae los indices creados en sim_scores y crea una lista de índices (i[0]) solamente si el índice no es igual al índice de la película consultada (idx). 
    # Esto excluye el índice de la película consultada de la lista de índices de películas similares, != idx] esto indica eso, es decir que no es igual al indice de la peli indicada.
    
    # Obtener la lista de títulos de películas similares
    similar_movie_titles = df_top_4['title'].iloc[similar_movie_indices].tolist()
    
    return similar_movie_titles 

In [158]:
recomendacion('toy story')

['Toy Story 2',
 'Toy Story 3',
 'Small Fry',
 'Hawaiian Vacation',
 'In a Heartbeat']

In [159]:
recomendacion('wonder woman')

['Fantastic Beasts and Where to Find Them',
 'Pokémon: Spell of the Unknown',
 'Highlander',
 'The Last Legion',
 'Doctor Strange']